## [Haegeman11](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.107.070601)
Time-Dependent Variational Principle for Quantum Lattices. J. Haegeman, J. I. Cirac, T. J. Osborne, I. Pižorn, H. Verschelde and F. Verstraete. *Phys. Rev. Lett.* **2011**, *107*, 070601.

[A practical guide](https://scipost.org/10.21468/SciPostPhysLectNotes.7): Tangent-Space Methods for Uniform Matrix Product States. L. Vanderstraeten, J. Haegeman, F. Verstraete. *SciPost Phys. Lect. Notes*, **2019**, 7,  

In [17]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.sparse.linalg import LinearOperator, eigs, bicgstab
import scipy.linalg
from opt_einsum import contract

# size of A
D = 256
d = 3
# step size
dtau = 0.1

# S=1 spin operators
sz = np.array([[1, 0, 0], [0, 0, 0], [0, 0, -1]])
sp = np.sqrt(2) * np.array([[0, 1, 0], [0, 0, 1], [0, 0, 0]])
sm = np.sqrt(2) * np.array([[0, 0, 0], [1, 0, 0], [0, 1, 0]])

sz1 = np.kron(sz, np.identity(d))
sz2 = np.kron(np.identity(d), sz)
sp1 = np.kron(sp, np.identity(d))
sp2 = np.kron(np.identity(d), sp)
sm1 = np.kron(sm, np.identity(d))
sm2 = np.kron(np.identity(d), sm)

J = 1

h = ( sz1 @ sz2 + 1/2 * sp1 @ sm2 + 1/2 * sm1 @ sp2).reshape(d, d, d, d)
orig_h = h.copy()

A = np.random.rand(D, d, D) - 0.5
energy = np.inf

while True:
    # round imaginary part to zero to reduce computational cost
    A = A.real
    
    # find l, r
    E_oper = LinearOperator((D**2, D**2), lambda x: contract("acd, bce, de -> ab", A, A, x.reshape(D, D)))
    E_oper_T = LinearOperator((D**2, D**2), lambda x: contract("ab, acd, bce -> de", x.reshape(D, D), A, A))
    # largest real part. Sometimes complex eigenvalue has very large norm but it's not the desired eigenvalue.
    evals, evecs = eigs(E_oper, k=1, which="LR")
    r = evecs[:, 0].real
    _, evecs = eigs(E_oper_T, k=1, which="LR")
    l = evecs[:, 0].real
    # normalize
    A /= np.sqrt(evals[0].real)
    r *= 1 / (l @ r)

    l = l.reshape(D, D)
    r = r.reshape(D, D)
    
    # energy expectation
    old_energy = energy
    energy = contract("ab, ace, egi, bdf, fhj, cgdh, ij", l, A, A, A, A, orig_h, r)
    if abs(old_energy - energy) < 1e-6:
        break
    h = (orig_h.reshape(d**2, d**2) - np.eye(d**2) * energy).reshape(d, d, d, d)
    print(energy)

    # solve L_h, and R_h
    I_minus_tilde_E = LinearOperator((D**2, D**2), lambda x: x - E_oper.dot(x) + x @ l.flatten() * r.flatten())
    I_minus_tilde_E_T = LinearOperator((D**2, D**2), lambda x: x - E_oper_T.dot(x) + x @ r.flatten() * l.flatten())
    L_h, info1 = bicgstab(I_minus_tilde_E_T, contract("ab, ace, egi, bdf, fhj, cgdh -> ij", l, A, A, A, A, h).ravel())
    R_h, info2 = bicgstab(I_minus_tilde_E, contract("ceg, gik, dfh, hjl, eifj, kl -> cd", A, A, A, A, h, r).ravel())
    assert info1 == info2 == 0
    L_h, R_h = L_h.reshape(D, D), R_h.reshape(D, D)
    
    # compute F
    F = contract("ab, acd, de -> bce", L_h, A, r) + \
        contract("ab, ace, egi, fhj, cgdh, ij -> bdf", l, A, A, A, h, r) + \
        contract("ab, ace, egi, bdf, cgdh, ij -> fhj", l, A, A, A, h, r) + \
        contract("ab, ace, de -> bce", l, A, R_h)
    
    # compute V_L. Note that sqrtm(L) could be complex so V_L could be complex.
    V_L = scipy.linalg.null_space(np.tensordot(scipy.linalg.sqrtm(l), A, axes=1).reshape(-1, D).T).reshape(D, d, D * (d-1))
    
    # compute time derivative
    l_half_inv = scipy.linalg.inv(scipy.linalg.sqrtm(l))
    r_inv = scipy.linalg.pinv(r)
    A_deriv = contract("ab, ce, bdh, efh, adg, gi -> cfi", l_half_inv, l_half_inv, V_L.conj(), V_L, F, r_inv)
    A_norm = np.linalg.norm(A)
    A_deriv_norm = np.linalg.norm(A_deriv)
    print(A_norm, A_deriv_norm, np.linalg.norm(F))
    
    # Euler step
    A = A - dtau * (A_norm / A_deriv_norm) * A_deriv


-0.0020629686453602636
15.990945872591896 21.86908329702029 0.108412031637631
-0.13578197916467102
16.024438472715605 22.067037877350455 0.1119338941672961
-0.27161517272681396
16.09751277586545 22.359798279538587 0.11829488675043255
-0.40846093112912923
16.206126331935586 22.7290638576859 0.12803047714787147
-0.5445969303828923
16.347366440481 23.101090139852722 0.1418771250206564
-0.6774816983456166
16.518557155580382 23.39968811786038 0.16174692732477444
-0.8038978517619397
16.71636069634428 23.59105767977156 0.19168289261261715
-0.920221789848721
16.936363818197187 23.70484495123543 0.2383267503490728
-1.022956715070156
17.173146310592692 23.825778277361565 0.3105788440555028
-1.109898542260033
17.420686577629915 24.09181274770582 0.416124496062812
-1.1812156840776136
17.672732722280756 24.70044597903067 0.5493317829984224
-1.2387124375486234
17.923135161516722 25.67569289239042 0.6666462153457644
-1.2835954537351006
18.167959490258077 26.62357600042982 0.6977641618167724
-1.316271

$D=1024$ result: $e=-1.4014840389712$